This notebook produces the example diagrams of steel flows in the paper. To get it set up, see [README](README.md).

In [ ]:
from networkx.readwrite import json_graph
import json

from sankeyview import *
from sankeyview.jupyter import show_sankey

Load the dataset:

In [ ]:
dataset = Dataset.from_csv('steel_flows.csv', dim_process_filename='steel_dim_process.csv')

Load some useful hierarchies for describing the data:

In [ ]:
with open('steel_hierarchies.json', 'rt') as f:
    json_trees = json.load(f)
    trees = {k: json_graph.node_link_graph(t) for k, t in json_trees.items()}

material_shape = Hierarchy(trees['material_shape'], 'material_shape')
func = Hierarchy(trees['function'], 'function')
sector = Hierarchy(trees['sector'], 'sector')

# Shortcut for partitioning based on process id
def p(*values):
    return Partition.Simple('process', values)

## Figure 9(a)

In [ ]:
partition_sectors = Partition.Simple('target.sector', [
    (k.replace(' e', '\ne'), sector._leaves_below(k))
    for k in ['Vehicles', 'Construction', 'Industrial equipment', 'Metal Goods']
])

partition_products = Partition.Simple('process.function', [
    (k.capitalize(), func._leaves_below(k + ' products') + func._leaves_below('fab ' + k))
    for k in ['flat', 'long', 'cast']
])

partition_cast_products = Partition([
    Group('Hot rolled', [('material_type', ['hot rolled steel'])]),
    Group('Cast iron', [('material_type', ['cast iron'])]),
    Group('Cast steel', [('material_type', ['cast steel'])])
])

order1 = [
    [['inputs', 'scrap sources', 'wpt_fab_scrap'], [], [], []],
    [['steelmaking'], ['forming scrap collection'], [], []],
    [['cold forming', 'wpt_cast_prods'], ['scrap2'], [], ['loss']],
    [['products'], [], ['fab scrap'], []],
    [['manufacturing2'], [], [], []],
    [['manufacturing1'], [], [], []],
]

nodes1 = {
    'inputs':         ProcessGroup(func('inputs'), title='Iron ore'),

    'steelmaking':    ProcessGroup(func('scrap prep', 'reduction', 'steelmaking', 'casting', 'hot forming'),
                                   title='Steelmaking\n& hot forming'),

    'cold forming':   ProcessGroup(func('cold forming', 'galvanising', 'organic coating'),
                                   title='Cold rolling\n& coating'),
                             
    'products':       ProcessGroup(func('intermediate products', 'fabrication'), partition_products,
                                   title='Fabrication by type'),
    
    'fab scrap':      ProcessGroup(func('scrap'), direction='L', title=''),
    
    'manufacturing2': Waypoint(partition_sectors),
    'manufacturing1': ProcessGroup(func('manufacturing'), title='Manufacturing'),
    
    'wpt_cast_prods': Waypoint(partition_cast_products, title=''),

    'loss':           ProcessGroup(('reduction loss', 'steelmaking loss', 'casting loss', 'forming loss'),
                                   title='Losses'),
    
    'forming scrap collection': ProcessGroup(('forming scrap collection',), direction='L', title='Forming scrap'),
    'scrap sources':            ProcessGroup(('end of life scrap',), title='EOL scrap'),
    
    'wpt_fab_scrap':  Waypoint(title='Fabrication scrap'),
    'scrap2':         Waypoint(title='', direction='L'),
}

bundles1 = [
    Bundle('inputs', 'steelmaking'),
    
    Bundle('steelmaking', 'cold forming'),
    
    Bundle('steelmaking', 'products', waypoints=['wpt_cast_prods']),
    Bundle('cold forming', 'products'),
    Bundle('products', 'fab scrap'),
    Bundle('fab scrap', 'steelmaking', waypoints=['wpt_fab_scrap']),
    
    Bundle('steelmaking', 'loss', waypoints=[]),
    
    Bundle('steelmaking', 'forming scrap collection'),
    Bundle('cold forming', 'forming scrap collection', waypoints=['scrap2']),
    
    Bundle('forming scrap collection', 'steelmaking'),
    Bundle('scrap sources', 'steelmaking'),
    
    Bundle('products', 'manufacturing1', waypoints=['manufacturing2'])
]

sdd1 = SankeyDefinition(nodes1, bundles1, order1, flow_partition=dataset.partition('material_type'))

In [ ]:
show_sankey(sdd1, dataset, width=900, height=400,
            override_link_layout={
                "products^Flat-fab scrap^*-unknown": {'r0': 15, 'r1': 15},
            }
).auto_save_svg('steel_sankey_1.svg')

> *The layout overrides are currently experimental and undocumented features of the Sankey layout code.*

## Figure 9(b)

First, define several partitions used in the diagram:

In [ ]:
# Partition by material coating
partition_coating = Partition.Simple('material_coating', [
    ('Tin & organic',['tin', 'organic']),
    ('Galvanized',   ['galvanised']),
    ('Uncoated',     ['none']),
])

# Partition by material type
partition_type = Partition.Simple('material_type', [
    (k.capitalize(), [k])
    for k in ['cast steel', 'cast iron', 'cold rolled steel', 'hot rolled steel']
])

# Partition by material shape
partition_shapes = Partition.Simple('material_shape', [
    ('Cast products',    ['cast products']),
    ('Electrical sheet', ['electrical sheet']),
    ('Rod/bar',          ['rod/bar']),
    ('Coil',             ['coil']),
    ('Narrow strip',     ['narrow strip']),
    ('Plate',            ['plate']),
    ('Tube',             ['seamless tube', 'welded tube']),
    ('Bar',              ['bar']),
    ('Wire rod',         ['wire rod']),
    ('Sections',         ['light sections', 'heavy sections', 'rail']),
    ('Reinforcing bar',  ['reinforcing bar']),
])

# Material types are sorted alphanumerically -- force the order to be the same as partition_shapes
partition_shapes_ordered = Partition([
    Group('{:03d}_{}'.format(i, m.label), m.query)
    for i, m in enumerate(partition_shapes.groups)
])

# Partition by the larger product categories, based on which process made them
partition_products2 = Partition.Simple('source.function', [
    (k.capitalize(), [k + ' products'] + func._leaves_below('fab ' + k))
    for k in ['cast', 'flat', 'long']
])

# Partition by the high-level groups of sectors using the steel
partition_sectors2 = Partition.Simple('target.sector', [
    (k.replace(' e', '\ne'), sector._leaves_below(k))
    for k in ['Vehicles', 'Industrial equipment', 'Metal Goods', 'Construction']
])

Now the actual diagram definition:

In [ ]:
order2 = [
    [['products_coating']],
    [['products_type']],
    [['products_shape']],
    [['products_grouped']],
    [['manufacturing']],
]

nodes2 = {
    'products_coating': Waypoint(partition_coating, title='Coating'),
    'products_type':    Waypoint(partition_type, title='Steel type'),
    'products_shape':   Waypoint(partition_shapes),
    'products_grouped': Waypoint(partition_products2),
    'manufacturing':    ProcessGroup(func('manufacturing'), partition_sectors2, title='Manufacturing'),
}

bundles2 = [
    Bundle(Elsewhere, 'manufacturing',
           waypoints=['products_coating', 'products_type', 'products_shape', 'products_grouped']),
]

sdd2 = SankeyDefinition(nodes2, bundles2, order2, flow_partition=partition_shapes_ordered)

Set up the colours

In [ ]:
from palettable.colorbrewer import qualitative
import itertools
palette = qualitative.Pastel1_9.hex_colors
palette = {m.label: v for m, v in zip(partition_shapes_ordered.groups, itertools.cycle(palette))}

In [ ]:
show_sankey(sdd2, dataset, width=900, height=400, palette=palette, align_link_types=True
).auto_save_svg('steel_sankey_2.svg')